**TESTING A LLM IN SAGEMAKER** 

+ GO TO [HugginFace model](https://huggingface.co/MBZUAI/LaMini-T5-738M)
+ Deploy/Sagemaker

In [1]:
##install some libraries
!pip install transformers einops accelerate bitsandbytes

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 90.7 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 14.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.4/770.4 kB 18.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 126.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 115.6 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer , AutoModelForSeq2SeqLM
from transformers import pipeline
import torch
import base64


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [3]:
hf_model='MBZUAI/LaMini-T5-738M' ##model in huggingFace to use

In [4]:
tokenizer= AutoTokenizer.from_pretrained(hf_model)
base_model= AutoModelForSeq2SeqLM.from_pretrained(hf_model,device_map='auto',torch_dtype=torch.float32)

In [5]:
!pip install langchain #to import hugginF.

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 19.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.3 MB/s eta 0:00:00:00:01
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.4/381.4 kB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 20.0 MB/s eta 0:00:00
  Using cached multidict-6.0.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (114 kB)
  Using cached yarl-1.9.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (268 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.7/225.7 kB 45.4 MB/s eta 0:00:00
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [6]:
from langchain.llms import HuggingFacePipeline

In [7]:
def llm_pipeline():
    pipe = pipeline(
        'text2text-generation',
        model = base_model,
        tokenizer = tokenizer,
        max_length = 256,
        do_sample=True,
        temperature = 0.3,
        top_p = 0.95
    )
    local_llm = HuggingFacePipeline(pipeline=pipe)
    return local_llm

In [10]:
in_prompt='Write a shortly article on Transformers models and IA'

In [12]:
model=llm_pipeline()
gen_text=model(in_prompt)
print(f'input={in_prompt}')
print(f'response:{gen_text}')

input=Write a shortly article on Transformers models and IA
response:Transformers models are a beloved franchise that has been around for decades. The first Transformers model was released in 1977 and quickly became a fan favorite. The series has since become one of the most popular and iconic franchises in the world, with countless popular characters and stories. The first Transformers model was the Transformers X-Men, which was released in 1977. The first Transformers model was the Transformers X-Men, which was released in 1982. The first Transformers model was the Transformers X-Men, which was introduced in 1985. The Transformers X-


In [13]:
!pip uninstall -y sagemaker ##by the version

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Found existing installation: sagemaker 2.167.0
Uninstalling sagemaker-2.167.0:
  Successfully uninstalled sagemaker-2.167.0


In [14]:
!pip install sagemaker==2.168.0 ##version what we need

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 844.7/844.7 kB 11.5 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Using cached PyYAML-6.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (682 kB)
  Created wheel for sagemaker: filename=sagemaker-2.168.0-py2.py3-none-any.whl size=1151019 sha256=b789a38f3a9ff84b93992a9076a57109b99a52efc090ccd0fe82edb901d1014c
  Stored in directory: /home/ec2-user/.cache/pip/wheels/1b/ee/d8/568281f41590696106b2776946fb5abb609f5ad91ef8be6296
Successfully built sagemaker
  Attempting uninstall:

<span style="color:green; font-size:20px;">**NOW THE MODEL FOR THE ENDPOINT**</span>


In [15]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
# HF_TASK https://github.com/huggingface/hub-docs/blob/main/tasks/src/const.ts
hub = {
	'HF_MODEL_ID':hf_model,
	'HF_TASK': 'text2text-generation',
	'device_map': 'auto',
	'torch_dtype': 'torch.float32'

}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="0.8.2"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
# g5 it's ok but g4dn got CUDA kernel
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g4dn.xlarge",
	container_startup_health_check_timeout=300,
  )
  
# send request
predictor.predict({
	"inputs": in_prompt,
})

----------!

[{'generated_text': 'Transformers models have been a beloved part of the Transformers franchise for decades. The iconic characters'}]

In [18]:
query='write a short article on ligthing Network'

#hyperparams payload
payload={
        'inputs': query,
        'parameters':{
            'do_sample':True,
            'top_p':0.7,
            'temperature':0.3,
            'top_k':50,
            'max_new_tokens':512,
            'repetition_penalty':1.03} }


In [19]:
response=predictor.predict(payload)
print(response)

[{'generated_text': 'Lingthing Network is a leading provider of online education and training solutions. It offers a wide range of courses and programs that cater to different learning styles and interests. The network is designed to provide a flexible and affordable learning environment for students, teachers, and professionals. It also offers a wide range of resources and tools to help students learn and improve their skills. Lingthing Network is known for its high-quality educational resources and innovative learning tools. Its extensive library of courses and resources offers students a wide range of learning opportunities. Additionally, Lingthing Network offers a wide range of educational resources and'}]


<span style="color:orange; font-size:20px;">**TEST THE ENDPOINT**</span>


In [20]:
#go to console and get the enpoint of this Sagemaker
ENDPOINT='huggingface-pytorch-tgi-inference-2023-07-19-22-31-57-701'

In [22]:
import boto3

runtime=boto3.client('runtime.sagemaker')

response=runtime.invoke_endpoint(EndpointName=ENDPOINT, ContentType='application/json', Body=json.dumps(payload))
prediction=json.loads(response['Body'].read().decode('utf-8'))
final_result=prediction[0]['generated_text']
print(final_result)

Lingthing Network is a leading provider of online gaming and gaming services. It offers a wide range of products and services, including online gaming, gaming consoles, gaming consoles, and more. The network has a vast library of games and resources, including online games, video games, and mobile games. It also offers a range of gaming tools and software, including online gaming consoles, gaming consoles, and more. The Lingthing Network is known for its user-friendly interface, high-quality graphics, and advanced gaming features. It is also known for its user-friendly interface and intuitive navigation
